# Redes Recurrentes / Redes Neuronales Recurrentes y Prophet para Predicción de Series Temporales (Core)
Descripción
Redes Neuronales Recurrentes y Prophet para Predicción de Series Temporales (Core)

Descripción:

En este ejercicio, implementarás una red neuronal recurrente (RNN) utilizando LSTM para predecir valores futuros en una serie temporal. Además, se te pide que explores el uso de la herramienta Prophet, un modelo de predicción de series temporales desarrollado por Facebook, y compares los resultados obtenidos con los de la RNN.



Objetivo:

El objetivo es predecir los valores futuros de una serie temporal utilizando redes recurrentes y comparar su rendimiento con Prophet. Este ejercicio te permitirá explorar dos enfoques distintos para la predicción de series temporales.



Instrucciones:

1. Selección del dataset:
- Utiliza un dataset de series temporales de Kaggle, como el de «Air Quality Prediction» o cualquier otro relacionado con predicción de demanda, precios de acciones, etc. Puedes elegir el dataset que más se relacione con tus intereses.
- Enlace sugerido: [Air Quality Prediction](https://www.kaggle.com/competitions/air-quality-prediction)

2. Preprocesamiento de datos:
- Realiza una exploración inicial de la serie temporal, incluyendo la visualización de la tendencia y la estacionalidad si es aplicable.
- Realiza un escalado de los datos, ya que las redes neuronales recurrentes se benefician de que los datos estén en un rango similar.

3. Implementación de RNN con LSTM:
- Implementa una red LSTM para predecir la serie temporal. Utiliza una arquitectura simple con una capa LSTM y una capa densa para predecir el valor futuro de la serie temporal.
- Asegúrate de dividir los datos en conjuntos de entrenamiento y prueba, utilizando las primeras partes de la serie para entrenar el modelo y las últimas partes para probar la precisión de las predicciones.

4. Uso de Prophet:
- Instala y utiliza Prophet para predecir la misma serie temporal. Prophet es un modelo que maneja fácilmente tendencias y estacionalidades, por lo que es ideal para comparación con redes recurrentes.
- Entrena Prophet con los mismos datos que utilizaste para la red LSTM y realiza predicciones en el conjunto de prueba.

5. Comparación de resultados:
- Compara las predicciones de la red LSTM con las predicciones de Prophet. Utiliza métricas como el RMSE (Root Mean Squared Error) o el MAE (Mean Absolute Error) para medir la precisión de ambos modelos.
- Visualiza las predicciones de ambos modelos superpuestas a la serie temporal original para ver cuál se ajusta mejor a los datos.

6. Análisis y discusión:
- Discute cuál de los modelos ofrece mejores resultados en este caso particular. ¿Por qué crees que Prophet o la red LSTM tuvieron un mejor rendimiento?
- Reflexiona sobre las ventajas y desventajas de cada enfoque para la predicción de series temporales.

In [22]:
import pandas as pd
import numpy as np

df = pd.read_csv('/Users/mathiorti/Desktop/Me/Cursos/Data Analisis and Machine Learning/Data Sets/AirQuality.csv')
print(df.head())

   No  DEWP  TEMP    PRES cbwd   Iws  Is  Ir  AQI
0   1   -16  -4.0  1020.0   SE  1.79   0   0  129
1   2   -15  -4.0  1020.0   SE  2.68   0   0  148
2   3   -11  -5.0  1021.0   SE  3.57   0   0  159
3   4    -7  -5.0  1022.0   SE  5.36   1   0  181
4   5    -7  -5.0  1022.0   SE  6.25   2   0  138


In [23]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41757 entries, 0 to 41756
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   No      41757 non-null  int64  
 1   DEWP    41757 non-null  int64  
 2   TEMP    41757 non-null  float64
 3   PRES    41757 non-null  float64
 4   cbwd    41757 non-null  object 
 5   Iws     41757 non-null  float64
 6   Is      41757 non-null  int64  
 7   Ir      41757 non-null  int64  
 8   AQI     41757 non-null  int64  
dtypes: float64(3), int64(5), object(1)
memory usage: 2.9+ MB
None


In [24]:
print(df.describe())

                 No          DEWP          TEMP          PRES           Iws  \
count  41757.000000  41757.000000  41757.000000  41757.000000  41757.000000   
mean   20879.000000      1.750174     12.401561   1016.442896     23.866747   
std    12054.351932     14.433658     12.175215     10.300733     49.617495   
min        1.000000    -40.000000    -19.000000    991.000000      0.450000   
25%    10440.000000    -10.000000      2.000000   1008.000000      1.790000   
50%    20879.000000      2.000000     14.000000   1016.000000      5.370000   
75%    31318.000000     15.000000     23.000000   1025.000000     21.910000   
max    41757.000000     28.000000     42.000000   1046.000000    565.490000   

                 Is            Ir           AQI  
count  41757.000000  41757.000000  41757.000000  
mean       0.055344      0.194866     98.613215  
std        0.778875      1.418165     92.050387  
min        0.000000      0.000000      0.000000  
25%        0.000000      0.000000     

In [25]:
print(df.isnull().sum())

No      0
DEWP    0
TEMP    0
PRES    0
cbwd    0
Iws     0
Is      0
Ir      0
AQI     0
dtype: int64


In [26]:
print(df.duplicated().sum())

0


In [27]:
pip install prophet

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [28]:
# Empiezo el modelo

from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_absolute_error

#La columna id me parece innecesaria
df = df.drop('No', axis=1)

#Separo los datos en categóricos y numéricos
df_cat = df.select_dtypes(include=['object'])
df_num = df.select_dtypes(include=['number']).drop('AQI', axis = 1)

#Aplico One-Hot a las columnas categóricas
encoder = OneHotEncoder(sparse_output=False)
df_cat_encoded = encoder.fit_transform(df_cat)
df_cat_encoded = pd.DataFrame(df_cat_encoded, columns=encoder.get_feature_names_out(df_cat.columns))

#Aplico StandardScaler a las columnas numéricas
escaler = StandardScaler()
df_num_scaled = escaler.fit_transform(df_num)
df_num_scaled = pd.DataFrame(df_num_scaled, columns=df_num.columns)

#Ahora combino nuevamentas las categóricas y las numéricas y separo en X e Y
X = pd.concat([df_num_scaled, df_cat_encoded], axis = 1)
y = df['AQI'] #Mi variabe objetivo

# Creo las secuencias directamente
SEQ_LENGTH = 10  # Número de pasos de tiempo para cada secuencia
X_seq, y_seq = [], []

for i in range(len(X) - SEQ_LENGTH):
    X_seq.append(X[i:i + SEQ_LENGTH])
    y_seq.append(y[i + SEQ_LENGTH])

X_seq = np.array(X_seq)
y_seq = np.array(y_seq)

#Divido el dataset en entrenamiento y en prueba
X_train, X_test, y_train, y_test = train_test_split(X_seq, y_seq, test_size=0.2, random_state=42)

#Creo el modelo LSTM
model = Sequential([
    LSTM(50, activation='relu', input_shape=(SEQ_LENGTH, X_train.shape[2])),
    Dropout(0.2),
    Dense(1)  # Capa de salida para predecir un valor continuo
])

# Compilo el modelo
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Defino el callback de Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Entreno el modelo
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2, callbacks = [early_stopping])

# Evaluo el modelo en el conjunto de prueba
test_loss, test_mae = model.evaluate(X_test, y_test)
print(f'Test accuracy: {test_mae:.4f}')

# Calculo el MAE manualmente con mean_absolute_error
y_pred = model.predict(X_test)
manual_mae = mean_absolute_error(y_test, y_pred)
print(f'Test MAE (manual calculation): {manual_mae:.4f}')

Epoch 1/100


/Users/mathiorti/Library/Python/3.9/lib/python/site-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


835/835 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 9625.1221 - mae: 65.9579 - val_loss: 5364.2812 - val_mae: 52.1370
Epoch 2/100
835/835 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 5797.4941 - mae: 51.8180 - val_loss: 5166.4897 - val_mae: 48.0086
Epoch 3/100
835/835 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 5543.6191 - mae: 50.6674 - val_loss: 4844.3872 - val_mae: 47.1977
Epoch 4/100
835/835 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - loss: 5493.0220 - mae: 50.3740 - val_loss: 4745.1792 - val_mae: 46.2425
Epoch 5/100
835/835 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 5171.9282 - mae: 48.9201 - val_loss: 4762.9546 - val_mae: 50.0676
Epoch 6/100
835/835 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 5267.1318 - mae: 48.5412 - val_loss: 4536.1724 - val_mae: 46.9120
Epoch 7/100
835/835 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 4836.1802 - mae: 46.6952 - val_loss: 4989.4336 - val_mae: 44.9990
Epoch 8/100
835/835 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 4799.6240 - mae: 46.6384 - val_loss: 5193.0698 - val_mae: 53.2

In [31]:
from prophet import Prophet
from sklearn.metrics import mean_absolute_error
import pandas as pd

# Crear un dataframe con el formato adecuado para Prophet
df_prophet = df[['AQI']].copy()
df_prophet['ds'] = pd.date_range(start='1910-08-16', periods=len(df_prophet), freq='D')
df_prophet.rename(columns={'AQI': 'y'}, inplace=True)

# Verificar las primeras filas de df_prophet
print("Primeras filas de df_prophet:")
print(df_prophet.head())

# Dividir los datos en entrenamiento y prueba (80-20)
train_size = int(len(df_prophet) * 0.8)
train_df = df_prophet[:train_size]
test_df = df_prophet[train_size:]

# Crear y entrenar el modelo Prophet
model_prophet = Prophet()
model_prophet.fit(train_df)

# Realizar predicciones en el rango del conjunto de prueba
future = test_df[['ds']]
forecast = model_prophet.predict(future)

# Calcular el MAE
y_true = test_df['y'].values
y_pred = forecast['yhat'].values

mae = mean_absolute_error(y_true, y_pred)
print(f'\nTest MAE: {mae:.4f}')

# Mostrar las predicciones
print("\nPredicciones:")
print(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].head())



Primeras filas de df_prophet:
     y         ds
0  129 1910-08-16
1  148 1910-08-17
2  159 1910-08-18
3  181 1910-08-19
4  138 1910-08-20


12:49:43 - cmdstanpy - INFO - Chain [1] start processing
12:49:48 - cmdstanpy - INFO - Chain [1] done processing



Test MAE: 69.4639

Predicciones:
          ds       yhat  yhat_lower  yhat_upper
0 2002-01-30  93.331234  -16.298931  211.499846
1 2002-01-31  93.792394  -30.691734  202.284783
2 2002-02-01  93.692225  -27.914696  209.065995
3 2002-02-02  94.149359  -14.636319  211.579171
4 2002-02-03  94.565421  -20.551144  212.212146


# Conclusión
Prophet es muy bueno para datos con tendencias claras y estacionales.

LSTM puede capturar dependencias a largo plazo y patrones complejos en secuencias.

La red LSTM tiene un MAE de 41.2713, que es mucho menor que el MAE de Prophet de 69.4639. Entonces puede decir que la red LSTM es mejor en este caso porque produce predicciones más precisas según el MAE.